In [2]:
import os
import shutil
import xml.etree.ElementTree as ET
from ultralytics import YOLO
import random
import cv2
import pytesseract
import torch
import easyocr
import re
import time
import json
import re
import numpy as np
from collections import defaultdict, deque



In [2]:
import requests
import os
from requests_toolbelt.multipart.encoder import MultipartEncoder

def get_def_headers():
    return {
        "X-API-KEY": "123456"
    }

def get_base_api():
    return 'http://ec2-54-87-52-160.compute-1.amazonaws.com'


# Camera id use for identify location of their cameras
# please use these id below for testing:
# ad7de137-9287-402a-8b70-53684d96c88f: Future park, Rangsit, Thanyaburi, Prachatipat, Pathum Thani
# 2ec15a48-c819-494c-a807-5c0f41ebaf36: BTS Asok, Klongtoey Noei, Wattana, Bangkok
# 0e76998d-0590-4679-924a-049f92ab0b81: Lotus Laksi, Bangkhen, Anusawaree, Bangkok


# Send Notify API
def send_notify(license_plate: str, camera_id: str, upload_id: str):
    notify_api = get_base_api() + "/notify/v1/send"
    notify_json = {
        'licensePlate': license_plate,
        'cameraId': camera_id,
        'uploadId': upload_id
    }
    response = requests.post(notify_api, json=notify_json, headers=get_def_headers())
    return response.json()

# Upload Image API
def upload_image(image_path):
    upload_api = get_base_api() + "/media/v1/upload/image"
    filename = os.path.basename(image_path)

    with open(image_path, 'rb') as img_file:
        multipart_data = MultipartEncoder(
            fields={'image': (filename, img_file, 'image/jpeg')}
        )
        headers = get_def_headers()
        headers['Content-Type'] = multipart_data.content_type
        
        response = requests.post(upload_api, data=multipart_data, headers=headers)
    return response.json()

In [3]:
# Example Send Notify without Image
no_img_license_plate = "7กญ 3603 กรุงเทพมหานคร"
no_img_camera = "ad7de137-9287-402a-8b70-53684d96c88f"
response = send_notify(no_img_license_plate, no_img_camera, '')
print("Send Notify without Image response:", response)

Send Notify without Image response: {'notifyId': 'c2a1390c-45e3-414b-a6fb-a530a0409b6d', 'status': 'PENDING'}


In [4]:
# Example Send Upload Image
img_path = './runs/detect/predict4/car02.jpg'
upload_response = upload_image(img_path)
print("Send Upload response:", upload_response)

# Example Send Notify with Image
img_license_plate = '9กด 1881 กรุงเทพมหานคร'
img_camera = 'ad7de137-9287-402a-8b70-53684d96c88f'
img_upload = upload_response['uploadId']
notify_response = send_notify(img_license_plate, img_camera, img_upload)
print("Send Notify with Image response:", response)

Send Upload response: {'uploadId': '71bfe9c0-204f-4011-a1ed-6e81be8f386f', 'fileId': '5ff3a40a-513d-47c2-9d60-d0afacbf0e6f', 'filePath': '71bfe9c0-204f-4011-a1ed-6e81be8f386f/5ff3a40a-513d-47c2-9d60-d0afacbf0e6f.jpg', 'contentType': 'jpg', 'status': 'SUCCESS'}
Send Notify with Image response: {'notifyId': 'c2a1390c-45e3-414b-a6fb-a530a0409b6d', 'status': 'PENDING'}


In [ ]:
# -------------------------------
# ใช้สำหรับแบ่ง dataset เป็น train/val/test
# -------------------------------
images_dir = "data/images_all"      # โฟลเดอร์เก็บภาพทั้งหมด (เช่น D:\yolo\Project_2\data\images_all)
labels_dir = "data/labels_all"      # โฟลเดอร์เก็บ labels ทั้งหมด (เช่น D:\yolo\Project_2\data\labels_all)

output_base = "datasets/YOLO"       # โฟลเดอร์สำหรับ train/val/test
split_ratio = (0.7, 0.2, 0.1)       # train, val, test

# -------------------------------
# สร้างโฟลเดอร์ input ถ้าไม่มี
# -------------------------------
if not os.path.exists(images_dir):
    os.makedirs(images_dir)
    print(f"⚠️ สร้างโฟลเดอร์ '{images_dir}' แล้ว (ตอนนี้ว่างเปล่า) → กรุณาใส่ไฟล์ภาพก่อน")

if not os.path.exists(labels_dir):
    os.makedirs(labels_dir)
    print(f"⚠️ สร้างโฟลเดอร์ '{labels_dir}' แล้ว (ตอนนี้ว่างเปล่า) → กรุณาใส่ไฟล์ labels ก่อน")

# -------------------------------
# สร้างโฟลเดอร์ output (images + labels + split)
# -------------------------------
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(output_base, "images", split), exist_ok=True)
    os.makedirs(os.path.join(output_base, "labels", split), exist_ok=True)

# -------------------------------
# รวมรายชื่อไฟล์ภาพ
# -------------------------------
images = [f for f in os.listdir(images_dir) if f.lower().endswith((".jpg", ".png"))]

if not images:
    print(f"⚠️ ไม่พบไฟล์ภาพใน '{images_dir}' → กรุณาใส่ภาพก่อน")
    exit()

import random
random.shuffle(images)

n_total = len(images)
n_train = int(split_ratio[0] * n_total)
n_val = int(split_ratio[1] * n_total)
n_test = n_total - n_train - n_val

print(f"พบทั้งหมด {n_total} รูป → Train={n_train}, Val={n_val}, Test={n_test}")

# -------------------------------
# ฟังก์ชันย้ายไฟล์
# -------------------------------
import shutil
def move_files(file_list, split):
    for img_file in file_list:
        src_img = os.path.join(images_dir, img_file)
        dst_img = os.path.join(output_base, "images", split, img_file)

        # path ของ label (ชื่อเดียวกันแต่ .txt)
        label_file = os.path.splitext(img_file)[0] + ".txt"
        src_lbl = os.path.join(labels_dir, label_file)
        dst_lbl = os.path.join(output_base, "labels", split, label_file)

        # copy ภาพ
        shutil.copy(src_img, dst_img)

        # copy label ถ้ามี
        if os.path.exists(src_lbl):
            shutil.copy(src_lbl, dst_lbl)
        else:
            print(f"⚠️ ไม่มี label สำหรับ {img_file}")

# -------------------------------
# แบ่ง dataset
# -------------------------------
move_files(images[:n_train], "train")
move_files(images[n_train:n_train+n_val], "val")
move_files(images[n_train+n_val:], "test")

print("✅ แบ่ง dataset เสร็จแล้ว → อยู่ใน:", output_base)


พบทั้งหมด 1402 รูป → Train=981, Val=280, Test=141
✅ แบ่ง dataset เสร็จแล้ว → อยู่ใน: datasets/YOLO


In [ ]:
images_dir = "cartest"  
torch.cuda.empty_cache()


data_yaml  = "datasets/YOLO/data.yaml"
# -------------------------------


# เลือกโมเดล
model = YOLO("yolov8s.pt")  # small model, VRAM 8GB ยังรองรับ

# Train
model.train(
    data=data_yaml,
    epochs=100,
    imgsz=560,       # ขนาดภาพใหญ่ขึ้น
    batch=8,        
    name="plate_detector",
    augment=True,    # เพิ่มความแม่น
    device=0 ,         # ใช้ GPU
    half=True     # FP16 ลด VRAM
    
)

# ทดสอบโมเดล
results = model.predict(
    source=images_dir,
    save=True,
    conf=0.5
)
print("✅ Prediction finished, check runs/detect/plate_detector/")



New https://pypi.org/project/ultralytics/8.3.206 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.201  Python-3.10.18 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1060, 6144MiB)


AttributeError: module 'torch._C' has no attribute '_has_mps'

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Parking-watch simulator:
- รันกล้องต่อเนื่อง
- ตรวจหา "การมีอยู่" (presence) ของ plate ทุก 4 วินาที (YOLO only)
- ถ้าไม่มีรถ -> รีเซ็ตตัวจับเวลา
- ถ้ามีรถ (คันใดก็ได้) -> เริ่มจับเวลา
- ถ้าเจอรถต่อเนื่อง (คันใดก็ได้) และเวลาตั้งแต่เจอครั้งแรก >= 60s
  -> "ณ วินาทีนั้น" ค่อยสั่งอ่านทะเบียน (OCR)
  -> แคปภาพทั้งคัน + print บันทึก
  -> [NEW] ส่ง API แจ้งเตือน
- พิมพ์สถานะทั้งหมด (no DB, no network)
"""
import os
import re
import cv2
import time
import numpy as np
import torch
import difflib
from ultralytics import YOLO
from datetime import datetime
import base64  # <--- [NEW] Import
import requests # <--- [NEW] Import

# ---------------------------- fast_alpr import ----------------------------
try:
    from fast_alpr.alpr import ALPR, BaseOCR, OcrResult
except Exception as e:
    raise RuntimeError(f"❌ fast_alpr import failed: {e}\nInstall with: pip install fast-alpr")

# === [NEW] Import for API ===
try:
    from requests_toolbelt.multipart.encoder import MultipartEncoder
except Exception as e:
    raise RuntimeError(f"❌ requests_toolbelt import failed: {e}\nInstall with: pip install requests-toolbelt")
# ============================

# ---------------------------- จังหวัดไทย ----------------------------
thai_provinces = [
    "กรุงเทพมหานคร", "กระบี่", "กาญจนบุรี", "กาฬสินธุ์", "กำแพงเพชร",
    "ขอนแก่น", "จันทบุรี", "ฉะเชิงเทรา", "ชลบุรี", "ชัยนาท",
    "ชัยภูมิ", "ชุมพร", "เชียงราย", "เชียงใหม่", "ตรัง",
    "ตราด", "ตาก", "นครนายก", "นครปฐม", "นครพนม",
    "นครราชสีมา", "นครศรีธรรมราช", "นครสวรรค์", "นนทบุรี", "นราธิวาส",
    "น่าน", "บึงกาฬ", "บุรีรัมย์", "ปทุมธานี", "ประจวบคีรีขันธ์",
    "ปราจีนบุรี", "ปัตตานี", "พระนครศรีอยุธยา", "พังงา", "พัทลุง",
    "พิจิตร", "พิษณุโลก", "เพชรบุรี", "เพชรบูรณ์", "แพร่",
    "พะเยา", "ภูเก็ต", "มหาสารคาม", "มุกดาหาร", "แม่ฮ่องสอน",
    "ยะลา", "ร้อยเอ็ด", "ระนอง", "ระยอง",
    "ราชบุรี", "ลพบุรี", "ลำปาง", "ลำพูน", "เลย",
    "ศรีสะเกษ", "สกลนคร", "สงขลา", "สตูล", "สมุทรปราการ",
    "สมุทรสงคราม", "สมุทรสาคร", "สระแก้ว", "สระบุรี", "สิงห์บุรี",
    "สุโขทัย", "สุพรรณบุรี", "สุราษฎร์ธานี", "สุรินทร์", "หนองคาย",
    "หนองบัวลำภู", "อ่างทอง", "อำนาจเจริญ", "อุดรธานี", "อุตรดิตถ์",
    "อุทัยธานี", "อุบลราชธานี", "ประเทศไทย"
]

def correct_province(text):
    if not text:
        return None
    match = difflib.get_close_matches(text, thai_provinces, n=1, cutoff=0.3)
    return match[0] if match else None

# ---------------------------- OCR Engine ----------------------------
GPU_AVAILABLE = torch.cuda.is_available()

class EasyOCR_ALPR(BaseOCR):
    def __init__(self, lang_list=['th','en'], min_conf=0.1):
        import easyocr
        self.reader = easyocr.Reader(lang_list, gpu=GPU_AVAILABLE)
        self.min_conf = min_conf
        print(f"✅ EasyOCR_ALPR loaded (GPU={GPU_AVAILABLE})")

    def predict(self, image: np.ndarray):
        
        # === [FIX] แก้ไข Bug การประมวลผลซ้ำซ้อน ===
        # ถ้าภาพเป็น 3 channel (BGR) ให้ประมวลผล (Grayscale, CLAHE)
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            gray = cv2.bilateralFilter(gray, 6, 45, 45)
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
            processed_image = clahe.apply(gray)
        # ถ้าภาพเป็น 1 channel (Grayscale หรือ Binary) ให้ใช้ทันที
        else:
            processed_image = image
        # ==========================================

        ocr_results = self.reader.readtext(processed_image)
        results = []
        for r in ocr_results:
            bbox, text, conf = r if len(r)==3 else (None, r[1], 1.0)
            if conf >= self.min_conf and text.strip():
                results.append(OcrResult(text=text.strip(), confidence=float(conf)))
        return results

# ---------------------------- helpers (plate extraction + enhance) ----------------------------
def correct_common_thai_ocr_errors(text):
    # แก้ไขการจับคู่ที่ผิด
    corrections = {
        "ญณ": "ฌฌ", "ญญ": "ฌฌ", "ญ": "ฌ"
    }
    for wrong, right in corrections.items():
        text = text.replace(wrong, right)
    return text

def upscale_image(img, scale=2): 
    # ลด scale ลงหน่อย เพื่อให้ภาพไม่ใหญ่เกินไปในการ debug
    h, w = img.shape[:2]
    return cv2.resize(img, (w*scale, h*scale), interpolation=cv2.INTER_LANCZOS4)

def extract_province_from_text(text):
    candidates = re.split(r"[\s\n]+", text)
    for word in candidates:
        match = difflib.get_close_matches(word, thai_provinces, n=1, cutoff=0.25)
        if match:
            return match[0]
    return None

# === ปรับปรุง Regex ให้อนุญาตตัวอักษรภาษาอังกฤษ ===
def extract_thai_license_plate(text):
    cleaned = re.sub(r"[\n\r]+", " ", text)
    
    # อนุญาต ก-ฮ, a-z, A-Z, 0-9 (เผื่อ OCR อ่านไทยเป็นอังกฤษ)
    cleaned = re.sub(r"[^ก-ฮa-zA-Z0-9\s\-\.]", "", cleaned) 
    
    cleaned = re.sub(r"\s+", " ", cleaned).strip()
    
    # อนุญาต ก-ฮ และ a-zA-Z ในส่วนตัวอักษรของป้าย
    pattern = (
        r"([0-9]{0,2}\s*[ก-ฮa-zA-Z]{1,3}[\s\-\.]*\d{1,4})"     # หมายเลขทะเบียน
        r"[\s\n]*"
        r"(?:จังหวัด)?\s*([ก-ฮ]{2,20})?"                 # จังหวัด (optional, ยังคงเป็น ก-ฮ)
    )
    
    matches = re.findall(pattern, cleaned)
    results = []
    for plate, province in matches:
        plate = re.sub(r"[\s\-\.]", "", plate)
        province = province.strip() if province else None
        if province:
            best_match = difflib.get_close_matches(province, thai_provinces, n=1, cutoff=0.25)
            province = best_match[0] if best_match else None
        results.append({"plate": plate, "province": province})
    return results

# === ปรับปรุง Enhance Function (สำคัญมาก) ===
# ให้คืนค่าเป็น Grayscale แทน Binary
def enhance_for_ocr(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape)==3 else img.copy()
    
    # ใช้ Bilateral Filter เพื่อลด Noise แต่ยังคงขอบ (Edge) ไว้
    gray = cv2.bilateralFilter(gray, 5, 80, 80)
    
    # ใช้ CLAHE เพื่อเพิ่ม Contrast ในพื้นที่เล็กๆ
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(5,5))
    gray = clahe.apply(gray)
    
    # อาจจะเบลอนิดหน่อยเพื่อลด noise ที่อาจเพิ่มจาก CLAHE
    gray = cv2.GaussianBlur(gray, (3,3), 0)
    
    # ขยายภาพให้ใหญ่ขึ้น (สำคัญมากสำหรับ OCR)
    gray = cv2.resize(gray, None, fx=3.5, fy=3.5, interpolation=cv2.INTER_CUBIC)
    
    # ❌ ไม่ใช้ adaptiveThreshold ❌
    # ❌ ไม่ใช้ sharpen ❌
    
    # คืนค่าเป็น Grayscale image ที่ Enhance แล้ว
    return gray
# ===================================================

def safe_crop(img,x1,y1,x2,y2,pad=5):
    h,w=img.shape[:2]
    x1=max(0,x1-pad)
    y1=max(0,y1-pad)
    x2=min(w,x2+pad)
    y2=min(h,y2+pad)
    return img[y1:y2,x1:x2]

# === API Helper Function ===
def upload_image_base64(image_base64: str, filename: str = "image.jpg"):
    # ✅ แปลง base64 string -> bytes
    try:
        image_bytes = base64.b64decode(image_base64)
    except Exception as e:
        print(f"  ❌ Error decoding base64: {e}")
        return {"error": "base64 decode error", "uploadId": None}

    # ✅ เตรียม multipart/form-data
    multipart_data = MultipartEncoder(
        fields={'image': (filename, image_bytes, 'image/jpeg')}
    )

    headers = {
        "X-API-KEY": API_KEY,
        'Content-Type': multipart_data.content_type
    }

    # ✅ ส่ง request
    try:
        response = requests.post(API_UPLOAD_URL, data=multipart_data, headers=headers, timeout=10.0)
        if response.status_code == 201:
            return response.json()
        else:
            print(f"  ❌ Error uploading image: {response.status_code} {response.text}")
            return {"error": "upload failed", "uploadId": None}
    except Exception as e:
        print(f"  ❌ Exception during image upload: {e}")
        return {"error": str(e), "uploadId": None}
# ==================================

# ---------------------------- Initialize ALPR & YOLO ----------------------------
ocr_engine = EasyOCR_ALPR(lang_list=['th','en'])
alpr = ALPR(ocr=ocr_engine)   # ใช้งาน fast_alpr wrapper ของคุณ
YOLO_WEIGHTS = "runs/detect/plate_detector/weights/best.pt"  # เปลี่ยนให้ถูกต้อง
model = YOLO(YOLO_WEIGHTS)

# ---------------------------- Simulation / monitoring params ----------------------------

source = "demos/hard.mp4" # เปลี่ยนเป็นไฟล์วิดีโอ "car001.mp4" หรือ 0 สำหรับกล้อง
CHECK_INTERVAL = 3.0  # วินาที — ตรวจหา "presence" ทุก 3 วิ
LONG_STAY_THRESHOLD = 5.0 # วินาที — ถ้า "presence" นานเกิน 5 วิ => long stay
OUTPUT_DIR = "output_longstay"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === [NEW] API Constants ===
API_UPLOAD_URL = "http://ec2-54-87-52-160.compute-1.amazonaws.com/media/v1/upload/image"
API_NOTIFY_URL = "http://ec2-54-87-52-160.compute-1.amazonaws.com/notify/v1/send"
API_KEY = "123456"
# CAMERA_ID = "e8564251-3746-4599-846c-46ea8703b7a3" #ศูนย์ฝึก
CAMERA_ID = "2ec15a48-c819-494c-a807-5c0f41ebaf36" #อโศก
# ===========================

# === [LOGIC CHANGE] A tracking state based on "presence" ===
presence_first_seen = None       # datetime ของการพบ "รถ" (คันใดก็ได้) ครั้งแรก
presence_last_seen = None        # datetime ของการพบ "รถ" (คันใดก็ได้) ล่าสุด
last_detected_plate_text = "N/A" # สำหรับแสดงบนภาพสด

# ---------------------------- Main loop ----------------------------
cap = cv2.VideoCapture(source)
last_check = 0.0

print(f"✅ เริ่มระบบ (ตรวจ Presence ทุก {CHECK_INTERVAL} วิ, จอดนานเกิน {LONG_STAY_THRESHOLD} วิ). กด q เพื่อออก")

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ ไม่สามารถอ่าน frame ได้ (end of video or camera error). หยุดการทำงาน.")
        break

    now = time.time()
    display_frame = frame.copy() # ใช้สำเนาของ frame สำหรับการแสดงผล

    cv2.imshow("Camera Live", frame)
    # ตรวจในช่วง interval เท่านั้น
    if now - last_check < CHECK_INTERVAL:
        # ยังไม่ถึงเวลาเช็กใหม่ แต่ให้แสดงผล YOLO box ล่าสุดและป้ายที่อ่านได้
        if 'last_boxes' in locals() and len(last_boxes) > 0:
            for box in last_boxes:
                x1, y1, x2, y2 = map(int, box)
                cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(display_frame, last_detected_plate_text, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
        
        # แสดงเวลาที่ elapsed (ถ้ามีการจับเวลาอยู่)
        if presence_first_seen is not None:
            current_elapsed = (datetime.now() - presence_first_seen).total_seconds()
            cv2.putText(display_frame, f"Elapsed: {current_elapsed:.1f}s", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow("ALPR Live", display_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("ออกโดยผู้ใช้")
            break
        continue
    
    # === ส่วนนี้จะทำงานทุกๆ CHECK_INTERVAL วินาที ===
    last_check = now
    check_dt = datetime.now()
    print(f"\n--- ตรวจหา Presence เวลา: {check_dt.isoformat()} ---")

    # 1. รัน YOLO (Fast) เพื่อดูว่ามี "กล่อง" ป้ายทะเบียนหรือไม่
    results = model.predict(frame, conf=0.4, verbose=False) 
    boxes = results[0].boxes.xyxy.cpu().numpy() if len(results) > 0 else []
    last_boxes = boxes # เก็บ box ล่าสุดไว้สำหรับแสดงผลในช่วงพัก
    
    is_plate_present = len(boxes) > 0
    current_frame_plate_text = "N/A" # สำหรับแสดงบนภาพสดในรอบนี้

    # === ตรวจสอบตาม "Presence" (การมีอยู่) ===

    if is_plate_present:
        # 2. ถ้ามีรถ
        if presence_first_seen is None:
            # 2.1 ถ้าเพิ่งเจอครั้งแรก -> เริ่มจับเวลา
            presence_first_seen = check_dt
            presence_last_seen = check_dt
            print(f"  พบ Presence (มีรถ), เริ่มจับเวลา... {presence_first_seen.isoformat()}")
        else:
            # 2.2 ถ้าเจอต่อเนื่อง -> อัปเดตเวลา และเช็กว่านานเกินไปหรือยัง
            presence_last_seen = check_dt
            elapsed = (presence_last_seen - presence_first_seen).total_seconds()
            print(f"  พบ Presence ต่อเนื่อง (elapsed = {elapsed:.1f} s)")

            if elapsed >= LONG_STAY_THRESHOLD:
                # 3. [TRIGGER] ถ้านานเกิน Threshold -> "ณ จุดนี้" ค่อยสั่ง OCR (Slow)
                print(f"  (!) เวลาเกิน {LONG_STAY_THRESHOLD} s. ทำการอ่านป้ายทะเบียนเดี๋ยวนี้...")

                detected_plate = None
                detected_province = None
                
                # เก็บภาพ crop_enh ที่ดีที่สุดไว้
                best_crop_enh = None
                best_plate_text = None
                best_plate_province = None
                
                # === [MOVED] ย้ายโค้ด OCR มาไว้ตรงนี้ ===
                for i, box in enumerate(boxes): # เพิ่ม i สำหรับตั้งชื่อไฟล์
                    x1, y1, x2, y2 = map(int, box)
                    h_box = y2 - y1
                    extra = int(h_box * 0.8)
                    y2_expanded = min(frame.shape[0], y2 + extra)
                    crop = safe_crop(frame, x1, y1, x2, y2_expanded, pad=6)
                    if crop.size == 0: continue

                    crop_up = upscale_image(crop, scale=2)
                    
                    # ใช้ enhance_for_ocr ที่คืนค่า Grayscale
                    crop_enh = enhance_for_ocr(crop_up) 
                    
                    # รัน OCR ที่นี่ (เฉพาะเมื่อจำเป็น)
                    ocr_results = ocr_engine.predict(crop_enh) 
                    
                    texts = [r.text.strip() for r in ocr_results if hasattr(r, "text") and r.text.strip()]
                    combined_text = " ".join(texts)

                    # === [DEBUG] พิมพ์ผลลัพธ์ดิบจาก OCR ===
                    print(f"    -> OCR Raw Text (Box {i}): '{combined_text}'")
                    # ======================================

                    # ใช้ extract_thai_license_plate ที่แก้ไขแล้ว
                    plate_results = extract_thai_license_plate(combined_text) 
                    
                    if plate_results:
                        plate_info = plate_results[0]
                        current_detected_plate = plate_info.get('plate')
                        current_detected_province = plate_info.get('province') or extract_province_from_text(combined_text)
                        
                        # เลือกป้ายแรกที่อ่านได้เป็น best_plate
                        if current_detected_plate:
                            best_plate_text = current_detected_plate
                            best_plate_province = current_detected_province
                            best_crop_enh = crop_enh.copy() # เก็บสำเนาภาพไว้
                            print(f"    -> อ่านทะเบียนได้: {best_plate_text} {best_plate_province or ''}")
                            break # เอาแค่ป้ายแรกที่อ่านได้
                # === จบส่วน OCR ===

                last_detected_plate_text = best_plate_text if best_plate_text else "UNKNOWN"

                if best_plate_text:
                    # 4. ถ้าอ่านป้ายได้ -> บันทึก
                    timestamp_str = presence_last_seen.strftime('%Y%m%d_%H%M%S')
                    # บันทึกภาพเต็ม
                    fname_full = f"{OUTPUT_DIR}/{best_plate_text}_{timestamp_str}_full.jpg"
                    cv2.imwrite(fname_full, frame)
                    # บันทึกภาพป้ายทะเบียนที่ Enhance แล้ว (ตอนนี้เป็น Grayscale)
                    fname_ocr_crop = f"{OUTPUT_DIR}/{best_plate_text}_{timestamp_str}_ocr_crop.jpg"
                    cv2.imwrite(fname_ocr_crop, best_crop_enh)
                    
                    # === API INTEGRATION ===
                    print(f"  🚀 Preparing to send API notification for {best_plate_text}...")
                    
                    # 1. Encode image to base64 (ใช้ frame = ภาพเต็ม)
                    _, buffer = cv2.imencode('.jpg', frame) 
                    img_base64 = base64.b64encode(buffer).decode('utf-8')

                    # 2. Upload image
                    print(f"    - Uploading image...")
                    upload_response = upload_image_base64(img_base64, filename=fname_full)
                    upload_id = upload_response.get('uploadId') # ใช้ .get() เพื่อป้องกัน Error ถ้า key ไม่มี

                    if upload_id:
                        print(f"    - Image uploaded, uploadId: {upload_id}")
                        # 3. Send notification
                        payload = {
                            "licensePlate": best_plate_text + (f" {best_plate_province}" if best_plate_province else ""),
                            "uploadId": upload_id,
                            "cameraId": CAMERA_ID
                        }

                        print(f"    - Sending notification payload...")
                        try:
                            response = requests.post(
                                API_NOTIFY_URL,
                                headers={"X-API-KEY": API_KEY},
                                json=payload,
                                timeout=10.0
                            )
                            if response.status_code == 200:
                                print("    ✅ Notification API sent successfully.")
                            else:
                                print(f"    ❌ Notification API failed: {response.status_code} -> {response.text}")
                        except Exception as e:
                            print(f"    ⚠️ Exception during notification API call: {e}")
                    else:
                        print(f"    ❌ Skipping notification API call due to image upload failure.")
                    # ============================

                    # === แสดงผลภาพ Crop ที่อ่านได้ (เหมือนเดิม) ===
                    window_name = f"OCR Result: {best_plate_text}"
                    cv2.imshow(window_name, best_crop_enh)
                    cv2.waitKey(0) # รอให้ผู้ใช้กดปุ่ม
                    cv2.destroyWindow(window_name) # ปิดหน้าต่างนี้
                    # ==========================================

                    print(f"\n>>> LONG STAY DETECTED: {best_plate_text}")
                    print(f"    - first_seen: {presence_first_seen.isoformat()}")
                    print(f"    - last_seen : {presence_last_seen.isoformat()}")
                    print(f"    - elapsed   : {elapsed:.1f} seconds")
                    print(f"    - saved full image: {fname_full}")
                    print(f"    - saved OCR crop image: {fname_ocr_crop}\n")
                    
                    # <--- [NEW] แสดง CameraView และจบการทำงาน ---
                    print("  (!) ดำเนินการเสร็จสิ้น, จบการทำงาน")
                    cv2.imshow("Camera Live", frame) # แสดง CameraView ครั้งสุดท้าย
                    cv2.waitKey(1) # ให้เวลา window update
                    break # <--- จบ loop ทันที
                    # -----------------------------------------------
                    
                else:
                    # 4.1 ถ้าอ่านป้ายไม่ได้ (ณ วินาทีสุดท้าย)
                    timestamp_str = presence_last_seen.strftime('%Y%m%d_%H%M%S')
                    # บันทึกภาพเต็มแม้จะอ่านป้ายไม่ได้ก็ตาม
                    fname_full = f"{OUTPUT_DIR}/UNKNOWN_{timestamp_str}_full.jpg"
                    cv2.imwrite(fname_full, frame)
                    
                    print(f"\n>>> LONG STAY DETECTED (Unknown Plate)")
                    print(f"    - first_seen: {presence_first_seen.isoformat()}")
                    print(f"    - elapsed   : {elapsed:.1f} seconds")
                    print(f"    - (ไม่สามารถอ่านป้ายทะเบียนได้ในจังหวะสุดท้าย)")
                    print(f"    - saved full image (unknown plate): {fname_full}\n")
                    
                    # ถ้าต้องการเก็บ crop_enh ที่อ่านไม่ได้ด้วย (เพื่อ debug):
                    if len(boxes) > 0: # ถ้ามี box แต่ OCR อ่านไม่ได้
                        x1, y1, x2, y2 = map(int, boxes[0]) # เอา box แรก
                        h_box = y2 - y1
                        extra = int(h_box * 0.8)
                        y2_expanded = min(frame.shape[0], y2 + extra)
                        crop = safe_crop(frame, x1, y1, x2, y2_expanded, pad=6)
                        if crop.size > 0:
                            crop_up = upscale_image(crop, scale=2)
                            temp_crop_enh = enhance_for_ocr(crop_up) # ใช้ตัว Enhance ใหม่
                            fname_ocr_crop_unknown = f"{OUTPUT_DIR}/UNKNOWN_{timestamp_str}_ocr_crop.jpg"
                            cv2.imwrite(fname_ocr_crop_unknown, temp_crop_enh)
                            print(f"    - saved OCR crop image (for debug): {fname_ocr_crop_unknown}\n")
                            cv2.imshow(f"OCR Crop (UNKNOWN)", temp_crop_enh) # แสดงภาพให้เห็น
                            cv2.waitKey(0) # รอให้ปิดหน้าต่างก่อนไปต่อ
                            cv2.destroyWindow(f"OCR Crop (UNKNOWN)")
                            
                            # <--- แสดง CameraView และจบการทำงาน ---
                            print("  (!) ดำเนินการเสร็จสิ้น (ไม่พบป้าย), จบการทำงาน")
                            cv2.imshow("Camera Live", frame) # แสดง CameraView ครั้งสุดท้าย
                            cv2.waitKey(1) # ให้เวลา window update
                            break # <--- จบ loop ทันที
                            # -----------------------------------------------

                # 5. รีเซ็ตตัวจับเวลา เพื่อเริ่มรอบใหม่ (ส่วนนี้จะถูกข้ามไปถ้า break)
                print("  รีเซ็ตตัวจับเวลา...")
                presence_first_seen = None
                presence_last_seen = None
    
    else:
        # 6. ถ้าไม่มีรถในเฟรมนี้
        print("  ไม่พบ Presence (ไม่มีรถ)")
        if presence_first_seen is not None:
            # 6.1 ถ้ารถเพิ่งหายไป -> รีเซ็ตตัวจับเวลา
            print("  รถเป้าหมายหายไป รีเซ็ตตัวจับเวลา")
            presence_first_seen = None
            presence_last_seen = None
        last_detected_plate_text = "N/A" # รีเซ็ตข้อความป้ายทะเบียนที่แสดงบนภาพสด

    # === แสดงผลบนภาพสด (ทุกๆ รอบของ Main loop) ===
    # วาดกรอบป้ายทะเบียน (จาก YOLO ล่าสุด)
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        # แสดงป้ายทะเบียนที่อ่านได้ (ถ้ามี)
        if last_detected_plate_text != "N/A" and last_detected_plate_text != "UNKNOWN":
            cv2.putText(display_frame, last_detected_plate_text, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
        
    # แสดงเวลาที่ elapsed (ถ้ามีการจับเวลาอยู่)
    if presence_first_seen is not None:
        current_elapsed = (datetime.now() - presence_first_seen).total_seconds()
        cv2.putText(display_frame, f"Elapsed: {current_elapsed:.1f}s / {LONG_STAY_THRESHOLD:.1f}s", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
    else:
        cv2.putText(display_frame, "No Presence", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow("ALPR Live", display_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("ออกโดยผู้ใช้")
        break

# end main loopq
cap.release()
cv2.destroyAllWindows()

Using CPU. Note: This module is much faster with a GPU.


✅ EasyOCR_ALPR loaded (GPU=False)


2025-10-18 14:34:19.965 python[7088:10370004] 2025-10-18 14:34:19.965048 [W:onnxruntime:, coreml_execution_provider.cc:113 GetCapability] CoreMLExecutionProvider::GetCapability, number of partitions supported by CoreML: 7 number of nodes in the graph: 693 number of nodes supported by CoreML: 675
INFO:open_image_models.detection.core.yolo_v9.inference:Using ONNX Runtime with ['CoreMLExecutionProvider', 'AzureExecutionProvider', 'CPUExecutionProvider'] provider(s)
INFO:open_image_models.detection.pipeline.license_plate:Initialized LicensePlateDetector with model /Users/tanitsak.le/.cache/open-image-models/yolo-v9-t-384-license-plate-end2end/yolo-v9-t-384-license-plates-end2end.onnx


✅ เริ่มระบบ (ตรวจ Presence ทุก 3.0 วิ, จอดนานเกิน 5.0 วิ). กด q เพื่อออก

--- ตรวจหา Presence เวลา: 2025-10-18T14:34:21.864060 ---
  ไม่พบ Presence (ไม่มีรถ)

--- ตรวจหา Presence เวลา: 2025-10-18T14:34:24.800752 ---
  ไม่พบ Presence (ไม่มีรถ)

--- ตรวจหา Presence เวลา: 2025-10-18T14:34:27.805154 ---
  ไม่พบ Presence (ไม่มีรถ)

--- ตรวจหา Presence เวลา: 2025-10-18T14:34:30.823568 ---
  ไม่พบ Presence (ไม่มีรถ)
❌ ไม่สามารถอ่าน frame ได้ (end of video or camera error). หยุดการทำงาน.


: 